# Introduction

Choosing a furniture for your household is very important, especially if you are a new family and moving in to a new place to plant your roots.

But managing your budget is also important.

In this post I will use ML algorithms to predict prices for furniture on Jamia website.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [ ]:
df = pd.read_csv("/kaggle/input/furniture-price-prediction/Furniture Price Prediction.csv")

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.isna().sum()

In [ ]:
absent = df[df['price'].isnull()]
df.dropna(inplace=True)

## There are more non-rated furniture than rated furniture

In [ ]:
non_rated = df[df['rate'] == 0.0].copy()

In [ ]:
non_rated.head()

In [ ]:
non_rated['price'].plot.hist(bins=20)

In [ ]:
rated = df[df['rate'] > 0.0].copy()

In [ ]:
rated['price'].plot.hist(bins=15)

In [ ]:
## Count of non_rated vs rated
print(non_rated.shape)
print(rated.shape)

## Pre-processing of the dataset

So, basically there are lot of unique variables for names and types of furniture. As I suggested, they are not fit for Label Encoder. Hence, I decided to leave these features behind

The only features I find fit enough for training models are:
* rate
* delivery
* sale

However, sale feature needs additional pre-processing. We need to remove "%" sign

In [ ]:
def sales_process(x):
    if "%" in x:
        return int(x[:-1])
    
    return x

In [ ]:
df['sale'] = df['sale'].apply(sales_process)
absent['sale'] = absent['sale'].apply(sales_process)

In [ ]:
# selecting the most important features
df = df[['rate', 'delivery', 'sale', 'price']]

## Models training, hypermarameters tuning phase

In [ ]:
x = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42, test_size=0.2)

In [ ]:
def model_evaluate(model, name, plot=True):
    y_pred = model.predict(x_test)
    tit = name + "\nMAE:{}\nMSE:{}".format(mean_absolute_error(y_pred, y_test), mean_squared_error(y_pred, y_test))
    if plot:
        dd = pd.DataFrame(x_test, columns=['rate', 'delivery', 'sale'])
        dd['price'] = y_pred
        sns.pairplot(dd, x_vars=['rate', 'delivery', 'sale'], y_vars=['price'])
        plt.suptitle(tit)
        plt.tight_layout()
    else:
        print(tit)

In [ ]:
def model_train(model, name):
    model.fit(x_train, y_train)
    model_evaluate(model, name, False)
    return model

In [ ]:
lr = LinearRegression()
rfr = RandomForestRegressor(n_estimators=150, max_depth=60)
svr = SVR(kernel='linear', C=0.6)
abr = AdaBoostRegressor(n_estimators=45, learning_rate=0.3)
knr = KNeighborsRegressor(n_neighbors=10)
models = [lr, rfr, svr, abr, knr]
names = ['Linear Regression', 'Random Forest Regressor', 'Support Vector Regression',
        'Ada Boost Regressor', 'KNeighbors Regressor']

In [ ]:
mls = []
for i, j in zip(models, names):
    mls.append(model_train(i, j))
    print()

In [ ]:
dt = pd.DataFrame(x_test, columns=['rate', 'delivery', 'sale'])
dt['price'] = y_test

# Evaluate trained models

### I used Mean Absolute Error(MAE)

![MAE](https://www.codingprof.com/wp-content/uploads/2021/12/Formula_MeanAbsoluteError.png)

### and Mean Squared Error(MSE)

![MSE](https://cdn-media-1.freecodecamp.org/images/hmZydSW9YegiMVPWq2JBpOpai3CejzQpGkNG)

In [ ]:
sns.pairplot(dt, x_vars=['rate', 'delivery', 'sale'], y_vars=['price'])
plt.suptitle("Original")
plt.tight_layout()
for i, j in zip(mls, names):
    model_evaluate(i, j)

# Conclusion

## The best and most consistent models used are: KNeighborsRegressor with K=10 and Random Forest Regressor max_depth = 60 and n_estimators = 150

However, we need to keep a note that features used are ratings, delivery and sale.

There are also such factors as brands and type of furniture. However, I suggested that using Label Encoder would be wrong, considering there would be 500+ unique labels for the column of furniture type.